# Load the Data

In [1]:
# basic package
import csv
import glob
import pandas as pd
import matplotlib as plt
from tqdm import tqdm
import numpy as np
import random
from operator import itemgetter
import matplotlib.pyplot as plt
import warnings


# ml related 
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
political = pd.read_csv("drive/Shareddrives/ML_&_Econometrics/Merged/political_selected.csv")
undernourish = pd.read_csv("drive/Shareddrives/ML_&_Econometrics/Merged/undernourish_selected.csv")
# drinking = pd.read_csv("drive/Shareddrives/ML_&_Econometrics/Merged/drinking_selected.csv")

meaning_map = pd.read_csv("drive/Shareddrives/ML_&_Econometrics/Merged/final_new_meaning_A.csv")

# Preparatory Code

In [4]:
def map_code_to_meaning(mapping, code_no):
    return (mapping.loc[mapping['code']==code_no]).iloc[0].var_name

# print the total percetnage of missing in each dataset
def total_percentage_missing(df):
    return(np.count_nonzero(df.isna()) / df.size)

# drop the top N rows with most NAs
def drop_top_N_rows_with_most_NAs(df, N=300):
    if N/len(df)> 0.2:
      warnings.warn("Based on your speficied N, you are dropping more then 20% of the data")

    print("shape before drop", df.shape)
    dict_nas = {}
    for i in range(len(df)):
        percentage = total_percentage_missing(df.iloc[i])
        dict_nas[i] = percentage
    res = dict(sorted(dict_nas.items(), key = itemgetter(1), reverse = True)[:N])
    # print("here")
    top_NAs_rows = list(res.keys())
    # print(top_NAs_rows)
    df.drop(top_NAs_rows, axis=0, inplace=True)
    print("shape after drop ", df.shape)
    print("Missing data percentage ", total_percentage_missing(df) )
    return df

# split into two dataset by year (default=2017)
# fist one include that year, second one is year after that
def split_by_year(df, split_at = 2017):
    res1 = df.loc[df['Year']<= split_at]
    res2 = df.loc[df['Year']> split_at]
    return res1, res2

def print_all_coeff(list_coef, feature_name):
    sort_index = reversed(np.argsort(list_coef))
    list_of_lists = []

    # print(sort_index)
    for d in sort_index:
      for i in d:
          # print("[i]", feature_name[i])
          # print(i)
          # print(i)
          temp = int(feature_name[i])
          list_coef 
          # print("here", len(list_coef[0]))
          if list_coef[0][i] !=0.0:
              list_of_lists.append([round(list_coef[0][i],10), feature_name[i], map_code_to_meaning(meaning_map, temp)])
    return pd.DataFrame(list_of_lists, columns =['non_zero_coefficient', 'code', 'variable_name'])



# Linear Regression Pipeline Code

In [5]:
  # pipeline on returning the coefficient of lasso regression
# also returns the score of the regressions
def linear_pipeline(df, target_name = 'political', split_year = 2017):

    if target_name not in df.columns:
        raise ValueError("The input dataframe doesn't have the column: political")
    
    if 'Continent' in df.columns:
      df = df.drop(columns =['Continent'])

    # default split at 2017
    political_pre_2017, political_post_2017 = split_by_year(df, split_at = split_year)
    
    # Note, the variable names here is only names, y_politcal can be any dataframe
    # doesn't have to be political 
    y_political = political_pre_2017.pop(target_name)
    X_political = political_pre_2017.drop(columns = ['Year', 'Area Code'])

    y_political_test = political_post_2017.pop(target_name)
    X_political_test = political_post_2017.drop(columns = ['Year', 'Area Code'])

    
    feature_names = X_political_test.columns

    # scale the X
    scaler = StandardScaler()
    political_scaler_X = scaler.fit(X_political)
    X_political_scaled = political_scaler_X.transform(X_political)
    X_political_test_scaled = political_scaler_X.transform(X_political_test)

    # scale the y
    y_political = y_political.values.reshape(-1,1)
    y_political_test = y_political_test.values.reshape(-1,1)
    political_scaler_y = scaler.fit(y_political)
    y_political_scaled = political_scaler_y.transform(y_political)
    y_political_test_scaled = political_scaler_y.transform(y_political_test)
    
    # print shapes
    print("Training Shape:", X_political_scaled.shape)
    print("Testing Shape", X_political_test_scaled.shape)
    
    # Run LASSO
    reg = LinearRegression().fit(X_political_scaled, y_political_scaled)

    # evaluation
    print("score on training dataset", reg.score(X_political_scaled, y_political_scaled) )
    print("score on testing dataset", reg.score(X_political_test_scaled, y_political_test_scaled))
    y_train_pred = reg.predict(X_political_scaled) # predicting for training
    y_pred = reg.predict(X_political_test_scaled)  # predicting for testing
    print("R squared score on training", r2_score(y_political_scaled, y_train_pred))
    print("R squared score on testing", r2_score(y_political_test_scaled, y_pred))
    
    print("Mean Absolute Error on training", mean_absolute_error(y_political_scaled, y_train_pred))
    print("Mean Absolute Error on testing", mean_absolute_error(y_political_test_scaled, y_pred))
    res_df = print_all_coeff(reg.coef_, feature_names)
    return res_df

# Process Data Before Feeding in Pipeline: check missing data and fill in NAs

In [6]:
# this chunck can only be run once
political = drop_top_N_rows_with_most_NAs(df= political, N= 300)
undernourish = drop_top_N_rows_with_most_NAs(df= undernourish, N= 300)
# drinking = drop_top_N_rows_with_most_NAs(df= drinking, N= 500)



# fill NAs
political = political.fillna(0)
undernourish = undernourish.fillna(0)
# drinking = drinking.fillna(0)

shape before drop (3705, 1002)
shape after drop  (3405, 1002)
Missing data percentage  0.003907896395168547
shape before drop (3933, 1002)
shape after drop  (3633, 1002)
Missing data percentage  0.02336230374373741


# Linear Regression on Entire Political Dataset

In [7]:
linear_pipeline(political, target_name = 'political')



Training Shape: (3041, 999)
Testing Shape (364, 999)
score on training dataset 0.926627328479596
score on testing dataset -3.0712652128213484e+16
R squared score on training 0.926627328479596
R squared score on testing -3.0712652128213484e+16
Mean Absolute Error on training 0.21000319715077573
Mean Absolute Error on testing 21875503.12511677


,non_zero_coefficient,code,variable_name
0,-5.679217e+10,1757723802,Cattle Manure left on pasture that leaches (N ...
1,-5.651313e+10,105272386,"Chickens, layers Manure treated (N content)"
2,-5.542543e+10,105372380,"Chickens, broilers Manure left on pasture (N c..."
3,-3.947749e+10,105372381,"Chickens, broilers Manure applied to soils (N ..."
4,-3.796577e+10,1755723802,All Animals Manure left on pasture that leache...
...,...,...,...
994,3.796577e+10,1755723801,All Animals Manure left on pasture that volati...
995,3.958597e+10,105272381,"Chickens, layers Manure applied to soils (N co..."
996,4.200966e+10,1757723801,Cattle Manure left on pasture that volatilises...
997,5.708336e+10,105472381,Chickens Manure applied to soils (N content)


In [8]:
linear_pipeline(undernourish, target_name = 'undernourish')

Training Shape: (3055, 999)
Testing Shape (578, 999)
score on training dataset 0.8462174057653171
score on testing dataset -1.9081498126587196e+16
R squared score on training 0.8462174057653171
R squared score on testing -1.9081498126587196e+16
Mean Absolute Error on training 0.28996758179072696
Mean Absolute Error on testing 26021302.8420029


,non_zero_coefficient,code,variable_name
0,-1.002371e+11,67967245,Humid tropical forest Biomass burned (dry matter)
1,-8.814423e+10,19085610,Non-alcoholic Beverages Import Quantity
2,-7.398868e+10,67927245,Open shrubland Biomass burned (dry matter)
3,-3.780898e+10,67977245,Other forest Biomass burned (dry matter)
4,-2.269680e+10,19075610,Alcoholic Beverages Import Quantity
...,...,...,...
992,8.921259e+09,6825723113,All sectors with LULUCF Emissions (CO2eq) (AR5)
993,2.137766e+10,68257273,All sectors with LULUCF Emissions (CO2)
994,7.243510e+10,67937245,Closed and open shrubland Biomass burned (dry ...
995,1.035947e+11,18955610,Beverages Import Quantity
